In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

from enum import Enum
from typing import Any, List

In [6]:
pillbox = np.load('images/pillbox.npy')
test = np.load('images/test.npy')
test_line = np.load('images/line.npy')

In [7]:
def detect_epsu(roi, dest_img):
    dest_img = dest_img.copy()
    BLACK_BACKGROUND = cv2.cvtColor(dest_img, cv2.COLOR_RGB2GRAY)
    BLACK_BACKGROUND[BLACK_BACKGROUND == 0] = 255
    BLACK_BACKGROUND[BLACK_BACKGROUND != 0] = 255
    # Na kanale b modelu Lab są dobrze wyodrębnione żółte pigułki EPSU (wyraźnie jaśniejsze od pozostałych)
    GRAY = cv2.threshold(cv2.cvtColor(roi, cv2.COLOR_RGB2LAB)[..., 2], 167, 255, cv2.THRESH_BINARY)[1]
    CIRCLES = cv2.HoughCircles(GRAY, cv2.HOUGH_GRADIENT, 1, 20,
                                param1 = 30, param2 = 7,
                                minRadius = 3, maxRadius = 25)

    if CIRCLES is not None:
        CIRCLES = np.uint16(np.around(CIRCLES))
        for i in CIRCLES[0, :]:
            center = (i[0], i[1])
            cv2.circle(dest_img, center, 1, (0, 100, 100), 3)
            radius = i[2]
            cv2.circle(dest_img, center, radius, (255, 0, 0), 3)

    return dest_img


def detect_keto(roi, dest_img):
    dest_img = dest_img.copy()
    BLACK_BACKGROUND = cv2.cvtColor(dest_img, cv2.COLOR_RGB2GRAY)
    BLACK_BACKGROUND[BLACK_BACKGROUND == 0] = 255
    BLACK_BACKGROUND[BLACK_BACKGROUND != 0] = 255
    GRAY = cv2.cvtColor(roi, cv2.COLOR_RGB2HLS)[..., 2]
    CIRCLES = cv2.HoughCircles(GRAY, cv2.HOUGH_GRADIENT, 1, 25,
                                param1 = 30, param2 = 26,
                                minRadius = 40, maxRadius = 50)

    if CIRCLES is not None:
        CIRCLES = np.uint16(np.around(CIRCLES))
        for i in CIRCLES[0, :]:
            center = (i[0], i[1])
            cv2.circle(dest_img, center, 1, (0, 100, 100), 3)
            radius = i[2]
            cv2.circle(dest_img, center, radius, (0, 255, 0), 3)

    return dest_img


def detect_biox(roi, dest_img):
    dest_img = dest_img.copy()
    BLACK_BACKGROUND = cv2.cvtColor(dest_img, cv2.COLOR_RGB2GRAY)
    BLACK_BACKGROUND[BLACK_BACKGROUND == 0] = 255
    BLACK_BACKGROUND[BLACK_BACKGROUND != 0] = 255
    GRAY = roi[..., 0]
    CIRCLES = cv2.HoughCircles(GRAY, cv2.HOUGH_GRADIENT, 1, 20,
                                param1 = 5, param2 = 30,
                                minRadius = 26, maxRadius = 33)

    if CIRCLES is not None:
        CIRCLES = np.uint16(np.around(CIRCLES))
        for i in CIRCLES[0, :]:
            center = (i[0], i[1])
            cv2.circle(dest_img, center, 1, (0, 100, 100), 3)
            radius = i[2]
            cv2.circle(dest_img, center, radius, (255, 0, 0), 3)
            cv2.circle(BLACK_BACKGROUND, center, radius, (255, 255, 255))

    return dest_img


def detect_carbon(dest_img):
    dest_img = dest_img.copy()
    GRAY = cv2.cvtColor(dest_img, cv2.COLOR_RGB2HSV)[..., 2]
    
    _, t = cv2.threshold(GRAY, 45, 255, type = cv2.THRESH_BINARY)
    morph_img = cv2.morphologyEx(t, 
                                 cv2.MORPH_OPEN, 
                                 np.ones((3, 3), np.uint8), 
                                 iterations = 3)
    
    _, labels = cv2.connectedComponents(morph_img, connectivity = 4)
    sizes = np.bincount(labels.ravel())
    box_idx = np.argmax(sizes[1:]) + 1
    shine = morph_img.copy()
    shine[labels == box_idx] = 0
    
    removed_shine = morph_img.copy()
    removed_shine[shine == 255] = 0
    
    _, labels2 = cv2.connectedComponents(~removed_shine, connectivity = 4)
    sizes2 =  np.bincount(labels2.ravel())
    sizess = sizes2[2:]
    labelss = np.array(list(set(list(labels2.flatten())))[2:])
    dct = dict(zip(labelss, sizess))
    filtered_dct = {k: v for k, v in dct.items() if v > 10000}
    
    just_pills = ~removed_shine.copy()
    just_pills[~np.isin(labels2, list(filtered_dct.keys()))] = 0
    
    _, labels3 = cv2.connectedComponents(just_pills, connectivity = 4)

    if max(np.bincount(labels3.ravel())) > 15000:
        lab = cv2.cvtColor(dest_img, cv2.COLOR_RGB2LAB)[..., 2]
        t_lab = cv2.threshold(lab, 130, 255, cv2.THRESH_BINARY)[1]
        erode = just_pills.copy()
        erode[t_lab == 0] = 0
        eroded_pills = cv2.morphologyEx(erode, cv2.MORPH_CLOSE, np.ones((3, 3), np.uint8), iterations = 10)
        _, markers = cv2.connectedComponents(eroded_pills, connectivity= 4)
        
        color_full_image = just_pills.copy()
        cv2.watershed(cv2.merge([color_full_image] * 3), markers)
        color_full_image[markers == -1] = 0
        color_full_image = cv2.erode(color_full_image, kernel = np.ones((3, 3), np.uint8), anchor = (-1, -1), borderType = cv2.BORDER_REPLICATE)
        just_pills = color_full_image.copy()
        
    contours, _ = cv2.findContours(just_pills, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(dest_img, contours, -1, (255, 105, 180), 3)
        
    return dest_img